In [35]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append('../')
from benchmarks.inference_models import *

models = (
        Banana(),
        Gaussian(ndims=100, eigenvalues='Gamma', numpy_seed= rng_inference_gym_icg),
        GermanCredit(),
        Brownian(),
        #ItemResponseTheory(),
        #StochasticVolatility()
)

dfs = [pd.read_csv('../results' + model.name + str(model.ndims) + '0.csv') for model in models]

round = lambda x: np.round(x, 3)
#round = lambda x: x

def get_mclmc(df, num_windows):
    return round(df[(df['sampler'] == 'mclmc:st3') & (df['num_windows'] == num_windows)]['ess_avg'].item())

def get_nuts(df):
    return round(df[df['sampler'] == 'nuts']['ess_avg'].item())

df = pd.DataFrame(np.array([[get_mclmc(df, 1) for df in dfs], [get_mclmc(df, 2) for df in dfs], [get_nuts(df) for df in dfs]]).T, columns= ['MCLMC1', 'MCLMC2', 'NUTS'])
df.insert(0, 'model', ['Banana', 'Ill Conditioned Gaussian', 'Sparse Logistic Regression', 'Brownian Motion'])

display(df)
df_old = pd.DataFrame([['Sparse Logistic Regression', 0.006, 0.011], ['Stochastic Volatility', 0.0014, 0.006]], columns = ['model', 'MCLMC1', 'NUTS'])
display(df_old)
#df = df.style.format(index=False, precision=3)
#print(df.to_latex(index= False))

,model,MCLMC1,MCLMC2,NUTS
0,Banana,0.032,0.028,0.009
1,Ill Conditioned Gaussian,0.000,0.000,0.001
2,Sparse Logistic Regression,0.018,0.019,0.005
3,Brownian Motion,0.073,0.065,0.023


,model,MCLMC1,NUTS
0,Sparse Logistic Regression,0.0060,0.011
1,Stochastic Volatility,0.0014,0.006
